# Рекомендация тарифов

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

**Цель работы:** построить модель для задачи классификации, которая выберет подходящий тариф для клиентов.

**Ход работы:**
1. Открытие и изучение файла;
2. Разделение исходных данных на обучающую, валидационную и тестовую выборки;
3. Исследование качества разных моделей;
4. Проверка качества моделей на тестовой выборке;
5. Проверка модели на вменяемость.

## Откройте и изучите файл

Откроем и изучим файл с данными.

In [12]:
#импорт нужных библиотек и функций
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

In [13]:
#открытие файла, вывод первых 5 строк на экран
data = pd.read_csv('/datasets/users_behavior.csv')
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [14]:
#вывод информации о таблице
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Предоставленная таблица состоит из 3214 объектов, каждый из которых - информация о поведении одного пользователя за месяц. Пропусков в таблице нет. Известно:
* `сalls` — количество звонков,
* `minutes` — суммарная длительность звонков в минутах,
* `messages` — количество sms-сообщений,
* `mb_used` — израсходованный интернет-трафик в Мб,
* `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Разбейте данные на выборки

Разделим данные на обучающую, валидационную и тестовую выборки. Исходные данные разобъем в соотношении 3:1:1. Для этого импортируем `train_test_split` из модуля `sklearn.model_selection`, затем разделим выборку сначала на обучающую и валидационную в отношении 60% на 40%, а затем валидационную поделим на валидационную и тестовую пополам.

In [15]:
#деление данных на обучающую, валидационную и тестовую выборки
data_train, data_valid = train_test_split(data, test_size=0.4, random_state=12345)
data_valid, data_test = train_test_split(data_valid, test_size=0.5, random_state=12345)

#вывод размеров выборок
print(data_train.shape)
print(data_valid.shape)
print(data_test.shape)

(1928, 5)
(643, 5)
(643, 5)


Полученные выборки разобъем на признаки и целевой признак. Цель работы - построить модель, определяющую подходящий тариф для клиента, следовательно, `is_ultra` - целевой признак.

In [16]:
#отделяем целевой признак от остальных признаков
features_train = data_train.drop('is_ultra', axis=1)
target_train = data_train['is_ultra']

features_valid = data_valid.drop('is_ultra', axis=1)
target_valid = data_valid['is_ultra']

features_test = data_test.drop('is_ultra', axis=1)
target_test = data_test['is_ultra']

## Исследуйте модели

### Дерево решений

Обучим модель, меняя значение глубины от 1 до 5, и сравним значения точности на валидационной выборке.

In [5]:
best_model_dt = None
best_result = 0

#меняем значение max_depth от 1 до 5
for depth in range(1, 6):
    #обучаем модель 
    model_dt = DecisionTreeClassifier(max_depth=depth, random_state=12345)
    model_dt.fit(features_train, target_train)
    result = model_dt.score(features_valid, target_valid)
    if result > best_result:
        best_model_dt = model_dt
        best_result = result
        
    #вывод значений accuracy для каждого max_depth
    print('max_depth =', depth, 'accuracy =', result)

max_depth = 1 accuracy = 0.7542768273716952
max_depth = 2 accuracy = 0.7822706065318819
max_depth = 3 accuracy = 0.7853810264385692
max_depth = 4 accuracy = 0.7791601866251944
max_depth = 5 accuracy = 0.7791601866251944


### Случайный лес

Обучим модель, меняя значение глубины от 1 до 5 и количества деревьев от 10 до 50, и выведем значение точности лучшей модели на валидационной выборке.

In [6]:
best_model_rf = None
best_result = 0
best_depth = 0
best_est = 0

#меняем значение max_depth от 1 до 5 и значение n_estimators от 10 до 50 с шагом 10
for depth in range(1, 6):
    for est in range(10, 51, 10):
        #обучаем модель
        model_rf = RandomForestClassifier(max_depth=depth, n_estimators=est, random_state=12345)
        model_rf.fit(features_train, target_train)
        
        #считаем accuracy
        result = model_rf.score(features_valid, target_valid)
        
        #ищем лучшую модель
        if result > best_result:
            best_model_rf = model_rf
            best_result = result
            best_depth = depth
            best_est = est
            
#выводим параметры лучшей модели
print('accuracy лучшей модели =', best_result,
      'max_depth =', best_depth,
      'n_estimators =', best_est)

accuracy лучшей модели = 0.7947122861586314 max_depth = 5 n_estimators = 40


### Логистическая регрессия

Обучим модель и выведем значение точности на валидационной выборке.

In [7]:
#обучаем модель
model_lr = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model_lr.fit(features_train, target_train)

#выводим accuracy
print('accuracy =', model_lr.score(features_valid, target_valid))

accuracy = 0.7107309486780715


### Вывод

Самое высокое значение метрики accuracy у случайного леса: 0.795 при глубине 5 и количестве деревьев 40, затем идет дерево решений: 0.785 при глубине 3, самое низкое значение получилось у логистической регрессии: 0.711. 

## Проверьте модель на тестовой выборке

Проверим лучшую модель на тестовой выборке

In [9]:
#значение точности на тестовой выборке
best_model_rf.score(features_test, target_test)

0.7838258164852255

### Вывод

Получилось примерно такое же значение точности, как и на валидационной выборке: 0.784.

## Проверьте модели на адекватность

Проверим выбранную модель на вменяемость, сравним ее со случайной.

In [11]:
dummy_clf = DummyClassifier(random_state=12345)
dummy_clf.fit(features_train, target_train)
dummy_clf.score(features_test, target_test)

0.6842923794712286

Значение точности у случайной модели на тестовой выборке - 0.684, что меньше, чем у нашей модели, следовательно, она прошла проверку.

## Общий вывод

В ходе данной работы был изучен предоставленный файл, данные разделены на обучающую, валидационную и тестовую выборки. Было исследовано качество разных моделей, модели были проверены на тестовой выборке. Выбор был остановлен на модели, созданной с помощью алгоритма случайный лес с глубиной 5 и количеством деревьев 40, было получено значение точности 0.784. Также модель прошла проверку на вменяемость.